In [1]:
import pickle
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/data/data.csv')
df.head(20)

,url,label
0,diaryofagameaddict.com,bad
1,espdesign.com.au,bad
2,iamagameaddict.com,bad
3,kalantzis.net,bad
4,slightlyoffcenter.net,bad
5,toddscarwash.com,bad
6,tubemoviez.com,bad
7,ipl.hk,bad
8,crackspider.us/toolbar/install.php?pack=exe,bad
9,pos-kupang.com/,bad


In [0]:
def add_http(x):
  if(x[:4] == 'http'):
    return x
  return 'http://' + x

In [0]:
url_list = df['url'].map(add_http)
labels = df['label']

In [0]:
from urllib.parse import  urlparse

In [0]:
import re
def netloc_extractor(x):
  if(x[:4] != 'http'):
    x = 'http://' + x
  parsed_x = urlparse(x)
  netloc = str(parsed_x.netloc)
  netloc = netloc.replace(' ', '')
  netloc = netloc.replace('.', ' ')
  return netloc

In [0]:
def path_extractor(x):
  if(x[:4] != 'http'):
    x = 'http://' + x
  parsed = urlparse(x)
  path = str(parsed.path)
  path = path.replace(' ', '')
  path = path.replace('/', ' ')
  return path

In [0]:
netloc_list = df['url'].map(netloc_extractor)
path_list = df['url'].map(path_extractor)

Training using only Net Locations of the URL

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(netloc_list, labels, test_size=0.33)

In [0]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
ngram_range = (1, 4)

In [0]:
count_vect = CountVectorizer(ngram_range = ngram_range)
train_bow = count_vect.fit_transform(X_train)
val_bow = count_vect.transform(X_val)

In [0]:
weights = {
    'good':1,
    'bad':5
}

In [12]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='l2', C=1.0, class_weight=weights, max_iter=10000, n_jobs=-1)
model.fit(train_bow, y_train)

LogisticRegression(C=1.0, class_weight={'bad': 5, 'good': 1}, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=10000, multi_class='auto', n_jobs=-1, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
val_predictions = model.predict(val_bow)

In [14]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
print(accuracy_score(y_val, val_predictions))
print(f1_score(y_val, val_predictions, average='weighted'))
print(confusion_matrix(y_val, val_predictions))

0.9600443951165372
0.9592863575578532
[[ 21252   3925]
 [  1619 111958]]


In [0]:
tfidf_vect = TfidfVectorizer(ngram_range=ngram_range)
train_tfidf = tfidf_vect.fit_transform(X_train)
val_tfidf = tfidf_vect.transform(X_val)

In [0]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean = False)
x_train = sc.fit_transform(train_tfidf)
x_val = sc.transform(val_tfidf)

In [17]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='l2', C=1.0, class_weight=weights, max_iter=10000, n_jobs=-1)
model.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight={'bad': 5, 'good': 1}, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=10000, multi_class='auto', n_jobs=-1, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [18]:
val_predictions = model.predict(x_val)
print(accuracy_score(y_val, val_predictions))
print(f1_score(y_val, val_predictions, average='weighted'))
print(confusion_matrix(y_val, val_predictions))

0.8607751848595356
0.8727339131724589
[[24042  1135]
 [18183 95394]]


Adding Path to the netlocation to see if'it helps with the accuracy

In [0]:
total_address = []
for i in range(len(netloc_list)):
  if(path_list[i] != ''):
    add = netloc_list[i] + ' ' + path_list[i]
  else:
    add = netloc_list[i]
  total_address.append(add)

In [0]:
from sklearn.model_selection import train_test_split
import numpy as np
X_train, X_val, y_train, y_val = train_test_split(np.array(total_address), labels, test_size=0.33)

In [0]:
count_vect = CountVectorizer(ngram_range=ngram_range)
train_bow = count_vect.fit_transform(X_train)
val_bow = count_vect.transform(X_val)

In [22]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='l2', C=1.0, class_weight=weights, max_iter=10000, n_jobs=-1)
model.fit(train_bow, y_train)

LogisticRegression(C=1.0, class_weight={'bad': 5, 'good': 1}, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=10000, multi_class='auto', n_jobs=-1, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [23]:
val_predictions = model.predict(val_bow)
print(accuracy_score(y_val, val_predictions))
print(f1_score(y_val, val_predictions, average='weighted'))
print(confusion_matrix(y_val, val_predictions))

0.9755898929039883
0.9752649595550051
[[ 22387   2505]
 [   882 112980]]


In [0]:
tfidf_vect = TfidfVectorizer(ngram_range=ngram_range)
train_tfidf = tfidf_vect.fit_transform(X_train)
val_tfidf = tfidf_vect.transform(X_val)

In [0]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean = False)
x_train = sc.fit_transform(train_tfidf)
x_val = sc.transform(val_tfidf)

In [26]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='l2', C=1.0, class_weight=weights, max_iter=10000, n_jobs=-1)
model.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight={'bad': 5, 'good': 1}, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=10000, multi_class='auto', n_jobs=-1, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [27]:
val_predictions = model.predict(x_val)
print(accuracy_score(y_val, val_predictions))
print(f1_score(y_val, val_predictions, average='weighted'))
print(confusion_matrix(y_val, val_predictions))

0.9794240166049267
0.9790469825921917
[[ 22365   2527]
 [   328 113534]]


In [0]:
filename = 'best_acc_wtih.sav'
pickle.dump(model, open(filename, 'wb'))
filename = 'best_acc_with_tfidf.sav'
pickle.dump(tfidf_vect, open(filename, 'wb'))
filename = 'best_acc_with_sc.sav'
pickle.dump(sc, open(filename, 'wb'))

In [0]:
# for single image
def preprocess_url(x):
  netloc = netloc_extractor(x)
  path = path_extractor(x)
  if path != '':  final = netloc + ' ' + path
  else: final = netloc
  vec = tfidf_vect.transform([final])
  x_test = sc.transform(vec)
  return x_test

In [30]:
prediction = model.predict(preprocess_url('virus.com'))
prediction

array(['good'], dtype=object)

Removing the last part like com, ru, as they seem to have dominating effect on the result

In [0]:
import re
def netloc_extractor(x):
  if(x[:4] != 'http'):
    x = 'http://' + x
  parsed_x = urlparse(x)
  netloc = str(parsed_x.netloc)
  netloc = netloc.replace(' ', '')
  netloc = netloc.replace('.', ' ')
  lst = netloc.split(' ')
  final = lst[0]
  for i in range(1, len(lst) - 1):
    final = final + ' ' + lst[i]
  return final

In [0]:
netloc_list = df['url'].map(netloc_extractor)
path_list = df['url'].map(path_extractor)

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(netloc_list, labels, test_size=0.33)

In [0]:
count_vect = CountVectorizer(ngram_range = ngram_range)
train_bow = count_vect.fit_transform(X_train)
val_bow = count_vect.transform(X_val)

In [35]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='l2', C=1.0, class_weight=weights, max_iter=10000, n_jobs=-1)
model.fit(train_bow, y_train)

LogisticRegression(C=1.0, class_weight={'bad': 5, 'good': 1}, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=10000, multi_class='auto', n_jobs=-1, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [36]:
val_predictions = model.predict(val_bow)
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
print(accuracy_score(y_val, val_predictions))
print(f1_score(y_val, val_predictions, average='weighted'))
print(confusion_matrix(y_val, val_predictions))

0.9467763091514478
0.9447505790500204
[[ 19304   5877]
 [  1508 112065]]


In [0]:
tfidf_vect = TfidfVectorizer(ngram_range=ngram_range)
train_tfidf = tfidf_vect.fit_transform(X_train)
val_tfidf = tfidf_vect.transform(X_val)

In [0]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean = False)
x_train = sc.fit_transform(train_tfidf)
x_val = sc.transform(val_tfidf)

In [39]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='l2', C=1.0, class_weight=weights, max_iter=10000, n_jobs=-1)
model.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight={'bad': 5, 'good': 1}, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=10000, multi_class='auto', n_jobs=-1, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [40]:
val_predictions = model.predict(x_val)
print(accuracy_score(y_val, val_predictions))
print(f1_score(y_val, val_predictions, average='weighted'))
print(confusion_matrix(y_val, val_predictions))

0.8479827608573447
0.8619476331380486
[[24239   942]
 [20151 93422]]


In [0]:
total_address = []
for i in range(len(netloc_list)):
  if(path_list[i] != ''):
    add = netloc_list[i] + ' ' + path_list[i]
  else:
    add = netloc_list[i]
  total_address.append(add)

In [0]:
from sklearn.model_selection import train_test_split
import numpy as np
X_train, X_val, y_train, y_val = train_test_split(np.array(total_address), labels, test_size=0.33)

In [0]:
count_vect = CountVectorizer(ngram_range=ngram_range)
train_bow = count_vect.fit_transform(X_train)
val_bow = count_vect.transform(X_val)

In [44]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='l2', C=1.0, class_weight=weights, max_iter=10000, n_jobs=-1)
model.fit(train_bow, y_train)

LogisticRegression(C=1.0, class_weight={'bad': 5, 'good': 1}, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=10000, multi_class='auto', n_jobs=-1, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [45]:
val_predictions = model.predict(val_bow)
print(accuracy_score(y_val, val_predictions))
print(f1_score(y_val, val_predictions, average='weighted'))
print(confusion_matrix(y_val, val_predictions))

0.9698963633480837
0.9692582457229083
[[ 21755   3364]
 [   813 112822]]


In [0]:
filename = 'best_acc.sav'
pickle.dump(model, open(filename, 'wb'))
filename = 'best_acc_bow.sav'
pickle.dump(count_vect, open(filename, 'wb'))

In [0]:
tfidf_vect = TfidfVectorizer(ngram_range=ngram_range)
train_tfidf = tfidf_vect.fit_transform(X_train)
val_tfidf = tfidf_vect.transform(X_val)

In [0]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean = False)
x_train = sc.fit_transform(train_tfidf)
x_val = sc.transform(val_tfidf)

In [49]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='l2', C=1.0, class_weight=weights, max_iter=10000, n_jobs=-1)
model.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight={'bad': 5, 'good': 1}, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=10000, multi_class='auto', n_jobs=-1, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [50]:
val_predictions = model.predict(x_val)
print(accuracy_score(y_val, val_predictions))
print(f1_score(y_val, val_predictions, average='weighted'))
print(confusion_matrix(y_val, val_predictions))

0.9286651195641207
0.9321171404430025
[[ 23969   1150]
 [  8748 104887]]


In [0]:
filename = 'best_bad.sav'
pickle.dump(model, open(filename, 'wb'))
filename = 'best_bad_tfidf.sav'
pickle.dump(tfidf_vect, open(filename, 'wb'))
filename = 'best_bad_sc.sav'
pickle.dump(sc, open(filename, 'wb'))

In [0]:
# for single image
def preprocess_url(x):
  netloc = netloc_extractor(x)
  path = path_extractor(x)
  if path != '':  final = netloc + ' ' + path
  else: final = netloc
  vec = tfidf_vect.transform([final])
  x_test = sc.transform(vec)
  return x_test

In [53]:
prediction = model.predict(preprocess_url('virus.com'))
prediction

array(['bad'], dtype=object)